In [ ]:
!git clone https://github.com/Xnehil/ZmBART.git

In [4]:
import os
os.chdir("ZmBART") 

#!git clone https://gist.github.com/f959c19ec18fee3ce2ff9b3b86b67c16.git
#!mv /kaggle/working/ZmBART/f959c19ec18fee3ce2ff9b3b86b67c16/convert_envyml_to_reqtxt /kaggle/working/ZmBART
#!python /kaggle/working/ZmBART/convert_envyml_to_reqtxt

In [ ]:
!cat requirements.txt | xargs -n 1 pip install
!pip install sentencepiece
!pip install onedrivedownloader

In [ ]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.CC25.tar.gz
!tar -xzvf mbart.CC25.tar.gz

In [ ]:
#from onedrivedownloader import download
#ln = "https://iith-my.sharepoint.com/personal/cs18resch11003_iith_ac_in/_layouts/15/download.aspx?UniqueId=f4bf857a-3037-49a6-abdf-d91738edae4d"
#download(url=ln, filename="checkpoint/zmBART.pt")
!pip install numpy==1.20.2

In [ ]:
import subprocess
import numpy as np

# Define variables
DATA = "../data"
TRAIN = "train"
VALID = "valid"
TEST = "test"
SRC = "isk"
TGT = "isk"
NAME = "isk-isk"
DEST = "../dataset/postprocess/NHG"
DICT = "mbart.cc25/dict.txt"

# Construct the command
command = (
    f"python -u preprocess.py "
    f"--source-lang {SRC} "
    f"--target-lang {TGT} "
    f"--trainpref {DATA}/{TRAIN}.spm "
    f"--validpref {DATA}/{VALID}.spm "
    f"--testpref {DATA}/{TEST}.spm "
    f"--destdir {DEST}/{NAME} "
    f"--thresholdtgt 0 "
    f"--thresholdsrc 0 "
    f"--srcdict {DICT} "
    f"--tgtdict {DICT} "
    f"--workers 70 "
    f"--fp16"
)

# Execute the command
subprocess.run(command, shell=True)

In [1]:
#Torch version

import torch

print(torch.__version__)

1.8.1+cu102


In [10]:
#Pretrain
import subprocess
import numpy as np

# Define the variables
PRETRAIN = "mbart.cc25"
langs = "ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN"
SRC = "isk"
TGT = "isk"
NAME = "isk-isk"
DATADIR = "../dataset/postprocess/aux/isk-isk"
SAVEDIR = "../checkpoint/checkpoint_ZmBART.pt"

# Construct the command
command = [
    "python", "-u", "train.py", DATADIR,
    "--arch", "mbart_large",
    "--task", "translation_from_pretrained_bart",
    "--source-lang", SRC,
    "--target-lang", TGT,
    "--criterion", "label_smoothed_cross_entropy",
    "--label-smoothing", "0.2",
    "--optimizer", "adam",
    "--adam-eps", "1e-06",
    "--lr-scheduler", "polynomial_decay",
    "--lr", "3e-05",
    "--warmup-updates", "2500",
    "--max-update", "100000",
    "--dropout", "0.3",
    "--max-tokens", "2048",
    "--update-freq", "2",
    "--save-interval", "1",
    "--save-interval-updates", "50000",
    "--keep-interval-updates", "10",
    "--seed", "222",
    "--log-interval", "100",
    "--restore-file", PRETRAIN,
    "--langs", langs,
    "--save-dir", SAVEDIR,
    "--tpu",
    "--skip-invalid-size-inputs-valid-test"
]

# Monkey patch np.float to np.float64
# np.float = np.float64
print("Running command:", " ".join(command))

# Monkey patch np.float to np.float64
np.float = np.float64

# Run the command with error handling
try:
    result = subprocess.run(command, check=True, capture_output=True, text=True)
    print("Command output:", result.stdout)
    print("Command error (if any):", result.stderr)
except subprocess.CalledProcessError as e:
    print("An error occurred while running the command.")
    print("Return code:", e.returncode)
    print("Output:", e.output)
    print("Error:", e.stderr)

Running command: python -u train.py ../dataset/postprocess/aux/isk-isk --arch mbart_large --task translation_from_pretrained_bart --source-lang isk --target-lang isk --criterion label_smoothed_cross_entropy --label-smoothing 0.2 --optimizer adam --adam-eps 1e-06 --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --max-update 100000 --dropout 0.3 --max-tokens 2048 --update-freq 2 --save-interval 1 --save-interval-updates 50000 --keep-interval-updates 10 --seed 222 --log-interval 100 --restore-file mbart.cc25 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN --save-dir ../checkpoint/checkpoint_ZmBART.pt --tpu --skip-invalid-size-inputs-valid-test


An error occurred while running the command.
Return code: 1
Output: 
Error: Traceback (most recent call last):
  File "train.py", line 362, in <module>
    cli_main()
  File "train.py", line 358, in cli_main
    distributed_utils.call_main(args, main)
  File "/home/harvymartinez16_gmail_com/Tesis/O3_modelos/ZmBART/fairseq/distributed_utils.py", line 180, in call_main
    import torch_xla.distributed.xla_multiprocessing as xmp
ModuleNotFoundError: No module named 'torch_xla'



In [2]:
!pip uninstall -y torch
!pip install torch==1.8.2+cpu torchvision==0.9.2+cpu -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install cloud-tpu-client==0.10 [https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl](https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl)


ERROR: torch_xla-2.0-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.


In [4]:
import torch
import torch_xla.core.xla_model as xm

dev = xm.xla_device()
t1 = torch.randn(3,3,device=dev)
t2 = torch.randn(3,3,device=dev)
print(t1 + t2)


ImportError: /home/harvymartinez16_gmail_com/miniconda3/envs/py37_ZmBART/lib/python3.7/site-packages/_XLAC.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZNK3c104Type14isSubtypeOfExtERKS0_PSo